# **Projeto Algoritmos Genéticos**
## Problema N-rainhas
### - Posicionar as *N* rainhas no tabuleiro sem que uma elimine a outra, ou seja, as peças não podem estar na mesma linha, coluna ou diagonal.
### - Especificações:


*   Quantidade de rainhas
*   Taxas: Mutação e Cruzamento
*   Tamanhos: população, eletismo e seleção
*   Critério de parada: Número de iterações e ou solução ótima






In [1]:
import random
from numpy.random import randint
from numpy.random import rand

In [2]:
#População gerada aleatoriamente
# size: tamanho do cromossomo
def random_chromosome(size):
    return [random.randint(1, size) for _ in range(size)]

In [3]:
# Geração de valores aleatórios para teste
# individual: Um cromossomo qualquer da população
# Função objetivo: número de pares de rainhas não-atacantes
# Exemplo: 8 rainhas 
# min = 0, max = 8 x 7/2 = 28
def fitness(individual):
    #FORMULA DE ATAQUES (COMBINAÇÃO SIMPLES DE 2 EM 2)
    h = len(individual)*(len(individual)-1)/2              
    #conferir se tem ataques na diagonal
    #primeiro FOR segura a coluna e o segundo percorre p fazer as combinações
    for i in range(0, len(individual)):
        for j in range(0, len(individual)):
            if j > i:
               if abs(i - j) == abs(individual[i] - individual[j]):
                  h -= 1
                  #conferir se tem ataques na horizontal
               if abs(individual[i] - individual[j]) == 0:
                  h -= 1 
    return h 

In [4]:
# Seleção por torneio
# population: População atual
# sizek: tamanho do torneio
def selection(population, sizeK):
	# seleciona um individuo aleatorio
	selection_ix = randint(len(population))
	for ix in randint(0, len(population), sizeK-1):
		# check if better (e.g. perform a tournament)
		if population[ix][1] > population[selection_ix][1]:
			selection_ix = ix
	return population[selection_ix][0]

In [5]:
#Cruzamento de 1 ponto aleatório
# p1 e p2: dois individuos da população
# r_cross: Taxa de cruzamento
def crossover(p1, p2, r_cross):
	#fazer uma copia dos pais(p1 e p2) para os filhos (c1 e c2)
	c1, c2 = p1.copy(), p2.copy()
	#aplicar probabilidade (caso r_cross = 0.9 -> 90% chance)
	if rand() < r_cross:
		#selecionar uma coluna (que não seja a ultima) para fazer a troca
		pt = randint(1, (len(p1)-1))		
		#realiza o cruzamento
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

In [6]:
# Mutação da posição de uma rainha
# individual: Um cromossomo qualquer da população
# r_mut: Taxa de mutação
def mutation(individual, r_mut): 
  for i in range(len(individual)):
    #possibilidade de fazer a mutação (se r_mut = 0.2 -> 20% de chance)
    if rand() < r_mut:
      #MUDAR DE POSIÇÃO UMA RAINHA
      individual[i] = random.randint(1, len(individual))

In [7]:
# Elitismo
#pop: População atual
#best_ind: Melhores individuos selecionados 

def elitism(pop, best_ind):
  #calcular quantos ataques por individuo
  scores = [fitness(p) for p in pop]
	#zipar a população com seu score
  newPop = []
  for i in zip(pop,scores):
      newPop.append(list(i))

  #juntar a população atual com os melhores individuos salvos anteriormente
  newPop = newPop + best_ind
  #reorganizar a população pelo seu score (do maior para o menor)
  newPop.sort(key = lambda newPop: newPop[:][1], reverse = True)
  #remover os x piores individuos
  #(x será o tamanho de individuos que a função best_ind vai "guardar")
  newPop = newPop[:-(len(best_ind))]
  #Limpa a população para poder receber a nova população sem seu score
  pop.clear() 
  for i in newPop:
    pop.append(i[0])  
  return pop

In [8]:
#Guardar os x melhores individuos da população, sendo x = sizeInd
def saveBestInd(pop,sizeInd):
  copyBest = [] #armazena os melhores individuos
  pop.sort(key = lambda pop: pop[:][1], reverse=True)
  copyBest = pop[:sizeInd].copy()
  return copyBest

In [9]:
#Mostra qual é a melhor solução da população no momento
def bestSolution(pop):
    pop.sort(key = lambda pop: pop[:][1], reverse=True)
    return pop[0][0],pop[0][1]

In [131]:
# Poderá ser utlizado qualquer quantidade de parâmetros
def genetic_algorithm(nqueens, n_iter, tam_pop, r_cross, r_mut, k, sizeBestInd, optimalSol):
	# Pop: data structure used:
	##### index [0] -> posição da linha (0,nqueens)
	##### index [1] -> score
	
	pop = [] #armazenar a população
	bestInd = [] #guardar os melhores indviduos (elitismo)

  #Criar população inicial
	tempPop = [random_chromosome(nqueens) for _ in range(tam_pop)]	
	
	#enumerar as gerações até o maximo de iterações(n_iter)
	for gen in range(n_iter):
			#calcular quantos ataques por individuo
			scores = [fitness(p) for p in tempPop]
			#zipar a população com seu score 
			for i in zip(tempPop,scores):
				pop.append(list(i))

      #apresentar qual é a melhor solução da geração atual
			ind, best_eval = bestSolution(pop)
			print("Geração: ",gen," Ind: ",ind," Fit: ",best_eval)
			#comparar melhor solução atual com a solução otima 
			#(caso for a melhor solução possivel, apresenta a geração e a solução e da um break)
			if best_eval == optimalSol:
				print(">>>>>> Melhor solução geração: %d" % gen)
				print(">>>>>> ",ind," - ",best_eval)
				break			
			#Salvar os melhores x individuos da população (ELITISMO)
			#(x = sizeBestInd)
			bestInd = saveBestInd(pop, sizeBestInd)					
			
			#realizar o torneio
			selected = [selection(pop, k) for _ in range(tam_pop)]

			#criar nova geração
			children = []
			#limpar população
			pop.clear()
			for i in range(0, tam_pop, 2):
				#selecionar os pais em pares
				p1, p2 = selected[i], selected[i+1]				
				#realizar o crossover
				for c in crossover(p1, p2, r_cross):
					#realizar a mutação
					mutation(c, r_mut)
					#guarda na nova população
					children.append(c)
		 
			#salva a população em tempPop
			tempPop = children
			#ELITISMO
			tempPop = elitism(tempPop, bestInd)			
	return tempPop 

In [11]:
def print_population(pop):
    print("--------------------------População final--------------------------")
    for i in pop:
      print("Cromossomo: ",i[0]," - Fit: ",i[1])

In [132]:
#PARÂMETROS BASE
tam_pop = 10
nqueens = 5
n_iter = 50

#POSSIBILIDADE DE PARÂMETROS PARA 8 RAINHAS
#tam_pop = 10
#nqueens = 8
#n_iter = 60

#POSSIBILIDADE DE ACONTECER CROSS
r_cross = 0.9

#POSSIBILIDADE DE ACONTECER MUTAÇÃO 
r_mut = 0.2

#TAMANHO DO TORNEIO
k = 3

#POPULAÇÃO NOVA 
newPop = []

#QUANTIDADE DE VALORES QUE SERÃO ARMAZENADOS (ELITISMO)
sizeBestInd = 2
optimalSol = nqueens * (nqueens-1)/2
print("----------- Solução otima = ", optimalSol, " -----------")

pop = genetic_algorithm(nqueens, n_iter, tam_pop, r_cross, r_mut, k, sizeBestInd, optimalSol)
fit = [fitness(chrom) for chrom in pop]
for i in zip(pop,fit):
  newPop.append(list(i))

print_population(newPop)  


----------- Solução otima =  10.0  -----------
Geração:  0  Ind:  [5, 1, 4, 2, 2]  Fit:  7.0
Geração:  1  Ind:  [5, 3, 4, 1, 5]  Fit:  7.0
Geração:  2  Ind:  [5, 1, 4, 1, 5]  Fit:  8.0
Geração:  3  Ind:  [5, 1, 1, 4, 2]  Fit:  9.0
Geração:  4  Ind:  [5, 1, 1, 4, 2]  Fit:  9.0
Geração:  5  Ind:  [5, 1, 1, 4, 2]  Fit:  9.0
Geração:  6  Ind:  [5, 1, 1, 4, 2]  Fit:  9.0
Geração:  7  Ind:  [5, 1, 1, 4, 2]  Fit:  9.0
Geração:  8  Ind:  [5, 1, 1, 4, 2]  Fit:  9.0
Geração:  9  Ind:  [5, 1, 1, 4, 2]  Fit:  9.0
Geração:  10  Ind:  [5, 1, 1, 4, 2]  Fit:  9.0
Geração:  11  Ind:  [5, 3, 1, 4, 2]  Fit:  10.0
>>>>>> Melhor solução geração: 11
>>>>>>  [5, 3, 1, 4, 2]  -  10.0
--------------------------População final--------------------------
Cromossomo:  [5, 3, 1, 4, 2]  - Fit:  10.0
Cromossomo:  [5, 1, 1, 4, 2]  - Fit:  9.0
Cromossomo:  [5, 1, 1, 4, 2]  - Fit:  9.0
Cromossomo:  [5, 1, 1, 4, 2]  - Fit:  9.0
Cromossomo:  [5, 1, 1, 4, 2]  - Fit:  9.0
Cromossomo:  [5, 1, 1, 4, 2]  - Fit:  9.0
Cromossomo

In [123]:
#geração 42 -> aleatorio
tam_pop = 100
nqueens = 10
n_iter = 50
r_cross = 0.9
r_mut = 0.3
k = 3
sizeBestInd = 5
newPop = []
optimalSol = nqueens * (nqueens-1)/2
print("----------- Solução otima = ", optimalSol, " -----------")

pop = genetic_algorithm(nqueens, n_iter, tam_pop, r_cross, r_mut, k, sizeBestInd, optimalSol)
fit = [fitness(chrom) for chrom in pop]
for i in zip(pop,fit):
  newPop.append(list(i))
#print_population(newPop)  

----------- Solução otima =  45.0  -----------
Geração:  0  Ind:  [7, 1, 4, 2, 8, 8, 3, 2, 5, 10]  Fit:  40.0
Geração:  1  Ind:  [10, 1, 4, 10, 7, 9, 5, 8, 5, 3]  Fit:  41.0
Geração:  2  Ind:  [10, 1, 4, 10, 7, 9, 5, 8, 5, 3]  Fit:  41.0
Geração:  3  Ind:  [6, 9, 2, 10, 3, 5, 8, 1, 3, 6]  Fit:  41.0
Geração:  4  Ind:  [3, 7, 2, 10, 5, 9, 8, 10, 3, 6]  Fit:  41.0
Geração:  5  Ind:  [3, 1, 4, 10, 1, 6, 2, 9, 6, 8]  Fit:  42.0
Geração:  6  Ind:  [3, 9, 4, 10, 1, 9, 2, 8, 2, 7]  Fit:  42.0
Geração:  7  Ind:  [3, 9, 4, 10, 1, 9, 2, 8, 2, 7]  Fit:  42.0
Geração:  8  Ind:  [3, 9, 4, 10, 1, 9, 2, 8, 2, 7]  Fit:  42.0
Geração:  9  Ind:  [1, 9, 6, 9, 2, 10, 4, 7, 5, 8]  Fit:  42.0
Geração:  10  Ind:  [3, 7, 2, 10, 5, 9, 8, 4, 9, 1]  Fit:  42.0
Geração:  11  Ind:  [1, 9, 6, 9, 2, 10, 4, 7, 5, 8]  Fit:  42.0
Geração:  12  Ind:  [3, 1, 4, 10, 1, 9, 2, 8, 2, 7]  Fit:  43.0
Geração:  13  Ind:  [3, 1, 4, 10, 1, 9, 2, 8, 2, 7]  Fit:  43.0
Geração:  14  Ind:  [3, 1, 4, 10, 1, 9, 2, 8, 2, 7]  Fit:  43.0
